## Avaliando [Crash Course](https://mxnet.apache.org/versions/master/api/python/docs/tutorials/getting-started/crash-course/3-autograd.html)


In [1]:
from mxnet import np,npx
from mxnet import autograd

npx.set_np()

In [2]:
x=np.array([[1,2],[3,4]])
x

[10:03:01] /home/mertins/Desenvolvimento/C/Terceiros/mxnet/src/storage/storage.cc:202: Using Pooled (Naive) StorageManager for CPU


array([[1., 2.],
       [3., 4.]])

In [3]:
x.attach_grad()

In [4]:
with autograd.record():
    y=2*x**2

In [5]:
y.backward()

Derivada de y=2x² => 4x

In [6]:
x.grad,y

(array([[ 4.,  8.],
        [12., 16.]]),
 array([[ 2.,  8.],
        [18., 32.]]))

In [7]:
with autograd.record():
    y=np.sum(2*x**2)
y.backward
x.grad

array([[ 4.,  8.],
       [12., 16.]])

In [8]:
with autograd.record():
    y=np.sum(2*x**2)
y.backward(retain_graph=True)
print(x.grad)
print("backward again because grad ratain")
y.backward()
print(x.grad)

try:
    y.backward()
except:
    print("não tem grad")

[[ 4.  8.]
 [12. 16.]]
backward again because grad ratain
[[ 4.  8.]
 [12. 16.]]
não tem grad


Custom MXNet ndarray operations

In [9]:
class MyFirstCustomOperation(autograd.Function):
    def __init__(self):
        super().__init__()

    def forward(self,x,y):
        return 2*x,2*x**2,2*y

    def backward(self, dx,dxy,dy):
        return x,y        

In [10]:
x=np.random.uniform(-1,1,(2,3))
y=np.random.uniform(-1,1,(2,3))
x.attach_grad()
y.attach_grad()
with autograd.record():
    z=MyFirstCustomOperation()
    z1,z2,z3=z(x,y)
    out=z1+z2+z3
out.backward()
print(np.array_equiv(x.asnumpy(),x.asnumpy()))
print(np.array_equiv(y.asnumpy(),y.asnumpy()))

True
True


In [11]:
def my_first_function(x):
    if autograd.is_training():
        return 4*x
    else:
        return x

In [12]:
y=my_first_function(x)
print(np.array_equiv(y.asnumpy,x.asnumpy()))
with autograd.record(train_mode=False):
    y=my_first_function(x)
y.backward()
print(x.grad)
with autograd.record(train_mode=True):
    y=my_first_function(x)
y.backward()
print(x.grad)


False
[[1. 1. 1.]
 [1. 1. 1.]]
[[4. 4. 4.]
 [4. 4. 4.]]


In [13]:
def my_second_function(x):
    with autograd.record():
        return(2*x)

In [14]:
y=my_second_function(x)
y.backward()
print(x.grad)

[[2. 2. 2.]
 [2. 2. 2.]]


In [15]:
y=my_second_function(x)
with autograd.record():
    z=my_second_function(y)+2
z.backward()
print(x.grad)

[[4. 4. 4.]
 [4. 4. 4.]]


In [16]:
def f(a):
    b=a*2
    while np.abs(b).sum()<1000:
        b=b*2
    if b.sum()>=0:
        c=b[0]
    else:
        c=b[1]
    return c

In [45]:
a=np.random.uniform(size=2)
a.attach_grad()
with autograd.record():
    c=f(a)
c.backward()
a,c,a.grad==c/a

(array([0.14335328, 0.53737324]), array(293.58752), array([ True, False]))

In [48]:
with autograd.record():
    y=3*x
    y.attach_grad()
    z=4*y+2*x
z.backward()
print(x.grad)
print(y.grad)

[[14. 14. 14.]
 [14. 14. 14.]]
[[4. 4. 4.]
 [4. 4. 4.]]


In [49]:
with autograd.record():
    y=3*x
    z=4*y+2*x
z.backward()
print(x.grad)
print(y.grad)

[[14. 14. 14.]
 [14. 14. 14.]]
None
